In [172]:
import pandas as pd
import requests
from io import StringIO
import re
import statistics
import time
import yfinance as yf
import numpy as np

### Question 1: [IPO] Withdrawn IPOs by Company Type

**What is the total withdrawn IPO value (in $ millions) for the company class with the highest total withdrawal value?**

From the withdrawn IPO list ([stockanalysis.com/ipos/withdrawn](https://stockanalysis.com/ipos/withdrawn/)), collect and process the data to find out which company type saw the most withdrawn IPO value.

#### Steps:
1. Use `pandas.read_html()` with the URL above to load the IPO withdrawal table into a DataFrame. 
   *It is a similar process to Code Snippet 1 discussed at the livestream.*    You should get **99 entries**. 
2. Create a new column called `Company Class`, categorizing company names based on patterns like:
   - “Acquisition Corp” or “Acquisition Corporation” → `Acq.Corp`
   - “Inc” or “Incorporated” → `Inc`
   - “Group” → `Group`
   - “Ltd” or “Limited” → `Limited`
   - “Holdings” → `Holdings`
   - Others → `Other`

  *  Order: Please follow the listed order of classes and assign the first matched value (e.g., for 'shenni holdings limited', you assign the 'Limited' class).

  * Hint: make your function more robust by converting names to lowercase and splitting into words before matching patterns.

3. Define a new field `Avg. price` by parsing the `Price Range` field (create a function and apply it to the `Price Range` column). Examples:
   - '$8.00-$10.00' → `9.0`  
   - '$5.00' → `5.0`  
   - '-' → `None`
4. Convert `Shares Offered` to numeric, clean missing or invalid values.
5. Create a new column:  
   `Withdrawn Value = Shares Offered * Avg Price` (**71 non-null values**)
6. Group by `Company Class` and calculate total withdrawn value.
7. **Answer**: Which class had the highest **total** value of withdrawals?


In [113]:
ipo_list_url = 'https://stockanalysis.com/ipos/withdrawn/'
headers = {
    'User-Agent': (
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
        'AppleWebKit/537.36 (KHTML, like Gecko) '
        'Chrome/58.0.3029.110 Safari/537.3'
    )
}


In [114]:
response = requests.get(ipo_list_url, headers=headers, timeout=10)

In [142]:
html_io = StringIO(response.text)
tables = pd.read_html(html_io)
tables

[   Symbol                Company Name      Price Range Shares Offered
 0    ODTX  Odyssey Therapeutics, Inc.                -              -
 1    UNFL      Unifoil Holdings, Inc.    $3.00 - $4.00        2000000
 2    AURN        Aurion Biotech, Inc.                -              -
 3    ROTR             PHI Group, Inc.                -              -
 4     ONE           One Power Company                -              -
 ..    ...                         ...              ...            ...
 95    FHP   Freehold Properties, Inc.                -              -
 96    CHO                Chobani Inc.                -              -
 97   IFIT  iFIT Health & Fitness Inc.  $18.00 - $21.00       30769231
 98   GLGX  Gerson Lehrman Group, Inc.                -              -
 99    HCG               hear.com N.V.  $17.00 - $20.00       16220000
 
 [100 rows x 4 columns],
      Date Symbol                 Name
 0  Jun 23   HCHL  Happy City Holdings
 1  Jun 24   FMFC     Kandal M Venture
 2  

In [143]:
ipo_df = tables[0]

In [144]:
ipo_df.shape

(100, 4)

In [122]:
from collections import Counter

def get_ngrams(text, n=2):
    words = text.lower().replace(',', '').split()
    return [' '.join(words[i:i+n]) for i in range(len(words)-n+1)]

cleaned_names = ipo_df['Company Name'].str.lower().str.replace(',', '')

single_words = cleaned_names.str.split().explode()
word_counts = Counter(single_words)

bigrams = cleaned_names.apply(lambda x: get_ngrams(x, n=2))
bigram_counts = Counter([item for sublist in bigrams for item in sublist])

print("Most common single words:")
print(pd.DataFrame(sorted(word_counts.items(), key=lambda x: x[1], reverse=True)[:25], 
                  columns=['Word', 'Count']))


Most common single words:
             Word  Count
0            inc.     51
1     acquisition     21
2           corp.     17
3           group     13
4         limited     12
5        holdings      9
6            ltd.      7
7     corporation      6
8    therapeutics      5
9             the      3
10        capital      3
11  international      3
12         health      3
13             iv      3
14        biotech      2
15          power      2
16        company      2
17    development      2
18            co.      2
19         global      2
20             ii      2
21              i      2
22         vector      2
23              &      2
24        odyssey      1


In [145]:
def categorize_company(name):
    if re.search(r'\b(Acquisition Corp|Acquisition Corporation)\b', name, re.IGNORECASE):
        return 'Acq.Corp'
    elif re.search(r'\b(Inc|Incorporated)\b', name, re.IGNORECASE):
        return 'Inc'
    elif re.search(r'\bGroup\b', name, re.IGNORECASE):
        return 'Group'
    elif re.search(r'\b(Ltd|Limited)\b', name, re.IGNORECASE):
        return 'Limited'
    elif re.search(r'\bHoldings\b', name, re.IGNORECASE):
        return 'Holdings'
    else:
        return 'Other'

ipo_df['Company Class'] = ipo_df['Company Name'].apply(categorize_company)
ipo_df['Company Class']

0       Inc
1       Inc
2       Inc
3       Inc
4     Other
      ...  
95      Inc
96      Inc
97      Inc
98      Inc
99    Other
Name: Company Class, Length: 100, dtype: object

In [146]:
ipo_df['Company Class'].value_counts()

Company Class
Inc         51
Acq.Corp    21
Limited     17
Other        6
Group        4
Holdings     1
Name: count, dtype: int64

In [147]:
def avg_price_range(val):
    try:
        return statistics.mean([float(s.replace('$', '')) if s else None for s in re.split(r'\s*-\s*', val)])
    except:
        return None

ipo_df['Avg. price'] = ipo_df['Price Range'].apply(avg_price_range)
ipo_df[['Avg. price', 'Price Range']]

,Avg. price,Price Range
0,NaN,-
1,3.5,$3.00 - $4.00
2,NaN,-
3,NaN,-
4,NaN,-
...,...,...
95,NaN,-
96,NaN,-
97,19.5,$18.00 - $21.00
98,NaN,-


In [148]:
ipo_df['Shares Offered'] = pd.to_numeric(ipo_df['Shares Offered'], errors='coerce')

ipo_df = ipo_df.dropna(subset=['Shares Offered'])

ipo_df['Shares Offered'] = ipo_df['Shares Offered'].astype(int)
ipo_df['Shares Offered']

/tmp/ipykernel_803708/3359939064.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ipo_df['Shares Offered'] = ipo_df['Shares Offered'].astype(int)


1      2000000
5      1400000
6       750000
7      2775000
8      3000000
        ...   
92    21000000
93    20000000
94    26100000
97    30769231
99    16220000
Name: Shares Offered, Length: 72, dtype: int64

In [149]:
ipo_df['Withdrawn Value'] = ipo_df['Shares Offered'] * ipo_df['Avg. price']
ipo_df['Withdrawn Value']

/tmp/ipykernel_803708/2720434111.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ipo_df['Withdrawn Value'] = ipo_df['Shares Offered'] * ipo_df['Avg. price']


1       7000000.0
5       7000000.0
6       3000000.0
7      24975000.0
8      15000000.0
         ...     
92    210000000.0
93    200000000.0
94    261000000.0
97    600000004.5
99    300070000.0
Name: Withdrawn Value, Length: 72, dtype: float64

In [150]:
agg = ipo_df.groupby('Company Class')['Withdrawn Value'].sum()
agg

Company Class
Acq.Corp    4.021000e+09
Group       3.378750e+07
Holdings    7.500000e+07
Inc         2.257164e+09
Limited     5.497346e+08
Other       7.679200e+08
Name: Withdrawn Value, dtype: float64

In [151]:
agg.idxmax()

'Acq.Corp'

In [152]:
agg.max() / 1_000_000

np.float64(4021.0)

In [153]:
def get_ipos(url) -> pd.DataFrame:
    """
    Fetch IPO data for the given year from stockanalysis.com.
    """
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/58.0.3029.110 Safari/537.3'
        )
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        # Wrap HTML text in StringIO to avoid deprecation warning
        # "Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object."
        html_io = StringIO(response.text)
        tables = pd.read_html(html_io)

        if not tables:
            raise ValueError(f"No tables found for year {year}.")

        return tables[0]

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except ValueError as ve:
        print(f"Data error: {ve}")
    except Exception as ex:
        print(f"Unexpected error: {ex}")

    return pd.DataFrame()


In [156]:
year = 2024
ipo_2024 = get_ipos(f"https://stockanalysis.com/ipos/{year}/")

In [157]:
ipo_2024

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"Dec 31, 2024",ONEG,OneConstruction Group Limited,$4.00,$5.00,25.00%
1,"Dec 27, 2024",PHH,"Park Ha Biological Technology Co., Ltd.",$4.00,$26.57,564.25%
2,"Dec 23, 2024",HIT,"Health In Tech, Inc.",$4.00,$0.62,-84.40%
3,"Dec 23, 2024",TDAC,Translational Development Acquisition Corp.,$10.00,$10.28,2.80%
4,"Dec 20, 2024",RANG,Range Capital Acquisition Corp.,$10.00,$10.21,2.10%
...,...,...,...,...,...,...
220,"Jan 18, 2024",CCTG,CCSC Technology International Holdings Limited,$6.00,$1.07,-82.17%
221,"Jan 18, 2024",PSBD,Palmer Square Capital BDC Inc.,$16.45,$14.50,-11.85%
222,"Jan 12, 2024",SYNX,Silynxcom Ltd.,$4.00,$2.07,-48.25%
223,"Jan 11, 2024",SDHC,Smith Douglas Homes Corp.,$21.00,$18.26,-13.05%


In [162]:
ipo_2024['IPO Date'] = pd.to_datetime(ipo_2024['IPO Date'])
ipo_2024 = ipo_2024[ipo_2024['IPO Date'] < '2024-06-01']

In [163]:
ipo_2024['IPO Price'] = pd.to_numeric(ipo_2024['IPO Price'].str.replace('$', ''), errors='coerce')
ipo_2024

/tmp/ipykernel_803708/3845361429.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ipo_2024['IPO Price'] = pd.to_numeric(ipo_2024['IPO Price'].str.replace('$', ''), errors='coerce')


,IPO Date,Symbol,Company Name,IPO Price,Current,Return
148,2024-05-31,NAKA,"Kindly MD, Inc.",NaN,$14.22,-
149,2024-05-23,BOW,Bowhead Specialty Holdings Inc.,17.00,$36.49,114.65%
150,2024-05-17,HDL,Super Hi International Holding Ltd.,19.56,$18.70,-4.40%
151,2024-05-17,RFAI,RF Acquisition Corp II,10.00,$10.60,6.00%
152,2024-05-15,JDZG,JIADE Limited,4.00,$0.26,-93.40%
...,...,...,...,...,...,...
220,2024-01-18,CCTG,CCSC Technology International Holdings Limited,6.00,$1.07,-82.17%
221,2024-01-18,PSBD,Palmer Square Capital BDC Inc.,16.45,$14.50,-11.85%
222,2024-01-12,SYNX,Silynxcom Ltd.,4.00,$2.07,-48.25%
223,2024-01-11,SDHC,Smith Douglas Homes Corp.,21.00,$18.26,-13.05%


In [165]:
ipo_2024['IPO Price'] = pd.to_numeric(ipo_2024['IPO Price'])
ipo_2024 = ipo_2024.dropna(subset=['IPO Price'])

/tmp/ipykernel_803708/31016230.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ipo_2024['IPO Price'] = pd.to_numeric(ipo_2024['IPO Price'])


In [190]:
ipo_2024['Symbol']

149     BOW
150     HDL
151    RFAI
152    JDZG
153     RAY
       ... 
220    CCTG
221    PSBD
222    SYNX
223    SDHC
224    ROMA
Name: Symbol, Length: 75, dtype: object

In [191]:
stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(ipo_2024['Symbol']):
  print(i,ticker)

  # Work with stock prices
  ticker_obj = yf.Ticker(ticker)

  # historyPrices = yf.download(tickers = ticker,
  #                    period = "max",
  #                    interval = "1d")
  historyPrices = ticker_obj.history(
                     period = "max",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in [1,3,7,30,90,365]:
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Close'] / historyPrices['Close'].shift(i)
  historyPrices['growth_future_30d'] = historyPrices['Close'].shift(-30) / historyPrices['Close']
  historyPrices['growth_252d'] = historyPrices['Close'] / historyPrices['Close'].shift(252)

  # Technical indicators
  # SimpleMovingAverage 10 days and 20 days
  historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  historyPrices['volatility'] =   historyPrices['Close'].rolling(30).std() * np.sqrt(252)

  # what we want to predict
  historyPrices['is_positive_growth_30d_future'] = np.where(historyPrices['growth_future_30d'] > 1, 1, 0)

  historyPrices['Sharpe'] = (historyPrices['growth_252d'] - 0.045) / historyPrices['volatility']

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)


0 BOW
1 HDL
2 RFAI
3 JDZG
4 RAY
5 BTOC
6 ZK
7 GPAT
8 PAL
9 SVCO
10 NNE
11 CCIX
12 VIK
13 ZONE
14 LOAR
15 MRX
16 RBRK
17 NCI
18 MFI
19 YYGH
20 TRSG
21 CDTG
22 CTRI
23 IBTA
24 MTEN
25 TWG
26 ULS
27 PACS
28 MNDR
29 CTNM
30 MAMO
31 ZBAO
32 BOLD
33 MMA
34 UBXG
35 IBAC
36 AUNA
37 BKHA
38 LOBO
39 RDDT
40 ALAB
41 INTJ
42 RYDE
43 LGCL
44 SMXT
45 VHAI
46 DYCQ
47 CHRO
48 UMAC
49 HLXB
50 MGX
51 TBBB
52 TELO
53 KYTX
54 PMNT
55 AHR
56 LEGT
57 ANRO
58 GUTS
59 AS
60 FBLG
61 AVBP
62 BTSG
63 HAO
64 CGON
65 YIBO
66 JL
67 SUGP
68 JVSA
69 KSPI
70 CCTG
71 PSBD
72 SYNX
73 SDHC
74 ROMA


In [174]:
def get_ticker_type(ticker:str, us_stocks_list, eu_stocks_list, india_stocks_list):
  if ticker in us_stocks_list:
    return 'US'
  elif ticker in eu_stocks_list:
    return 'EU'
  elif ticker in india_stocks_list:
    return 'INDIA'
  else:
    return 'ERROR'


In [196]:
stocks_df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Year,Month,...,growth_365d,growth_future_30d,growth_252d,SMA10,SMA20,growing_moving_average,high_minus_low_relative,volatility,is_positive_growth_30d_future,Sharpe
0,23.000000,24.270000,22.139999,23.799999,3335800,0.0,0.0,BOW,2024,5,...,NaN,1.092017,NaN,NaN,NaN,0,0.089496,NaN,1,NaN
1,24.260000,26.150000,23.980000,25.700001,990500,0.0,0.0,BOW,2024,5,...,NaN,0.998054,NaN,NaN,NaN,0,0.084436,NaN,0,NaN
2,25.850000,26.879999,25.075001,26.480000,555100,0.0,0.0,BOW,2024,5,...,NaN,1.001133,NaN,NaN,NaN,0,0.068165,NaN,1,NaN
3,26.440001,26.490000,25.500999,26.290001,302700,0.0,0.0,BOW,2024,5,...,NaN,0.987828,NaN,NaN,NaN,0,0.037619,NaN,0,NaN
4,27.209999,27.209999,25.500000,26.139999,200900,0.0,0.0,BOW,2024,5,...,NaN,1.037490,NaN,NaN,NaN,0,0.065417,NaN,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23316,2.870000,2.890000,2.560000,2.660000,123100,0.0,0.0,ROMA,2025,6,...,NaN,NaN,4.666667,3.1970,2.82155,1,0.124060,10.383358,0,0.445103
23317,2.840000,3.000000,2.640000,2.875000,63100,0.0,0.0,ROMA,2025,6,...,NaN,NaN,5.424529,3.1485,2.86180,1,0.125217,9.939668,0,0.541218
23318,2.850000,2.935000,2.790000,2.795000,10200,0.0,0.0,ROMA,2025,6,...,NaN,NaN,5.008960,3.0870,2.89955,1,0.051878,9.469241,0,0.524219
23319,2.883000,2.900000,2.730000,2.790000,33600,0.0,0.0,ROMA,2025,6,...,NaN,NaN,5.157116,2.9900,2.93555,1,0.060932,9.196931,0,0.555850


In [201]:
filtered_stocks = stocks_df[stocks_df.Date == pd.to_datetime('2025-06-06')]

In [202]:
filtered_stocks.describe()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Year,Month,Weekday,...,growth_365d,growth_future_30d,growth_252d,SMA10,SMA20,growing_moving_average,high_minus_low_relative,volatility,is_positive_growth_30d_future,Sharpe
count,75.000000,75.000000,75.000000,75.000000,7.500000e+01,75.0,75.0,75.0,75.0,75.0,...,0.0,0.0,71.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.0,71.000000
mean,16.617580,16.944701,16.253779,16.675501,1.392520e+06,0.0,0.0,2025.0,6.0,4.0,...,NaN,NaN,1.152897,16.345553,16.204357,0.493333,0.080612,19.228472,0.0,0.288285
min,0.000500,0.000600,0.000400,0.000600,0.000000e+00,0.0,0.0,2025.0,6.0,4.0,...,NaN,NaN,0.024970,0.000521,0.000584,0.000000,0.000000,0.004299,0.0,-0.079677
25%,1.260000,1.335000,1.230500,1.315000,2.669000e+04,0.0,0.0,2025.0,6.0,4.0,...,NaN,NaN,0.293422,1.326550,1.389400,0.000000,0.022842,1.421765,0.0,0.041215
50%,4.820000,4.959000,4.750000,4.810000,1.379000e+05,0.0,0.0,2025.0,6.0,4.0,...,NaN,NaN,0.758065,4.714000,4.698750,0.000000,0.045667,3.742442,0.0,0.083768
75%,21.645000,21.672501,21.152499,21.615000,5.123500e+05,0.0,0.0,2025.0,6.0,4.0,...,NaN,NaN,1.362736,20.419500,20.533250,1.000000,0.095427,19.271175,0.0,0.311507
max,113.629997,121.290001,112.769997,121.290001,4.401273e+07,0.0,0.0,2025.0,6.0,4.0,...,NaN,NaN,8.097413,110.690000,109.967500,1.000000,0.627119,188.035427,0.0,2.835668
std,25.939145,26.441644,25.284048,26.047440,5.288250e+06,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,1.406017,25.362283,24.993285,0.503322,0.108624,35.247872,0.0,0.519028


In [203]:
filtered_stocks['growth_252d'].median()

np.float64(0.7580645318618386)

In [204]:
filtered_stocks['Sharpe'].median()

np.float64(0.08376823003294499)

### Question 3: [IPO] ‘Fixed Months Holding Strategy’

**What is the optimal number of months (1 to 12) to hold a newly IPO'd stock in order to maximize average growth?**  
(*Assume you buy at the close of the first trading day and sell after a fixed number of trading days.*)


---

#### Goal:
Investigate whether holding an IPO stock for a fixed number of months after its first trading day produces better returns, using future growth columns.

---

#### Steps:

1. **Start from the existing DataFrame** from Question 2 (75 tickers from IPOs in the first 5 months of 2024).  

   Add **12 new columns**:  
   `future_growth_1m`, `future_growth_2m`, ..., `future_growth_12m`  
   *(Assume 1 month = 21 trading days, so growth is calculated over 21, 42, ..., 252 trading days)*  
   This logic is similar to `historyPrices['growth_future_30d']` from **Code Snippet 7**, but extended to longer timeframes.

2. **Determine the first trading day** (`min_date`) for each ticker.  
   This is the earliest date in the data for each stock.

3. **Join the data**:  
   Perform an **inner join** between the `min_date` DataFrame and the future growth data on both `ticker` and `date`.  
   ➤ You should end up with **75 records** (one per IPO) with all 12 `future_growth_...` fields populated.

4. **Compute descriptive statistics** for the resulting DataFrame:  
   Use `.describe()` or similar to analyze each of the 12 columns:  
   - `future_growth_1m`  
   - `future_growth_2m`  
   - ...  
   - `future_growth_12m`  

5. **Determine the best holding period**:  
   - Find the number of months **(1 to 12)** where the **average (mean)** future growth is **maximal**.  
   - This optimal month shows an uplift of **>1%** compared to all others.  
   - Still, the average return remains **less than 1** (i.e., expected return is less than doubling your investment).


In [208]:
stocks_min_date_df = stocks_df.groupby('Ticker')['Date'].min().reset_index()

In [215]:
stocks_min_date_df

,Ticker,Date
0,AHR,2024-02-07
1,ALAB,2024-03-20
2,ANRO,2024-02-02
3,AS,2024-02-01
4,AUNA,2024-03-22
...,...,...
70,YIBO,2024-01-25
71,YYGH,2024-04-22
72,ZBAO,2024-04-02
73,ZK,2024-05-10


In [213]:
for _,ticker in enumerate(ipo_2024['Symbol']):
    mask = stocks_df['Ticker'] == ticker
    close_prices = stocks_df.loc[mask, 'Close']
    
    for i in range(1, 13):
        stocks_df.loc[mask, f'growth_future_{i}m'] = close_prices.shift(-21 * i) / close_prices

In [214]:
stocks_df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Year,Month,...,growth_future_3m,growth_future_4m,growth_future_5m,growth_future_6m,growth_future_7m,growth_future_8m,growth_future_9m,growth_future_10m,growth_future_11m,growth_future_12m
0,23.000000,24.270000,22.139999,23.799999,3335800,0.0,0.0,BOW,2024,5,...,1.301681,1.167647,1.239076,1.461765,1.443698,1.357983,1.389076,1.722689,1.698319,1.533613
1,24.260000,26.150000,23.980000,25.700001,990500,0.0,0.0,BOW,2024,5,...,1.196498,1.090661,1.148249,1.392996,1.354086,1.280934,1.276654,1.640078,1.589883,1.445914
2,25.850000,26.879999,25.075001,26.480000,555100,0.0,0.0,BOW,2024,5,...,1.155589,1.048338,1.111027,1.361783,1.327039,1.233761,1.270015,1.535121,1.517749,1.408610
3,26.440001,26.490000,25.500999,26.290001,302700,0.0,0.0,BOW,2024,5,...,1.149106,1.053252,1.139977,1.378851,1.311525,1.248764,1.315709,1.559528,1.521491,1.430582
4,27.209999,27.209999,25.500000,26.139999,200900,0.0,0.0,BOW,2024,5,...,1.198164,1.071538,1.149579,1.383321,1.321729,1.236802,1.303367,1.595639,1.584545,1.432670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23316,2.870000,2.890000,2.560000,2.660000,123100,0.0,0.0,ROMA,2025,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23317,2.840000,3.000000,2.640000,2.875000,63100,0.0,0.0,ROMA,2025,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23318,2.850000,2.935000,2.790000,2.795000,10200,0.0,0.0,ROMA,2025,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23319,2.883000,2.900000,2.730000,2.790000,33600,0.0,0.0,ROMA,2025,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [217]:
df_first_trades = pd.merge(
    stocks_df,
    stocks_min_date_df,
    on=['Ticker', 'Date'],
    how='inner'
)

In [232]:
growth_cols = [f'growth_future_{i}m' for i in range(1, 13)]
mean_returns = df_first_trades[growth_cols].mean(skipna=True)
mean_returns 

growth_future_1m     0.927259
growth_future_2m     0.940544
growth_future_3m     0.833824
growth_future_4m     0.825086
growth_future_5m     0.803769
growth_future_6m     0.864186
growth_future_7m     0.847150
growth_future_8m     0.832982
growth_future_9m     0.881777
growth_future_10m    0.917943
growth_future_11m    0.882532
growth_future_12m    0.900859
dtype: float64

In [233]:
mean_returns.idxmax()


'growth_future_2m'

### Question 4: [Strategy] Simple RSI-Based Trading Strategy


**What is the total profit (in $thousands) you would have earned by investing $1000 every time a stock was oversold (RSI < 25)?**


---

#### Goal:
Apply a simple rule-based trading strategy using the **Relative Strength Index (RSI)** technical indicator to identify oversold signals and calculate profits.

---


#### Steps:

1. **Run the full notebook from Lecture 2 (33 stocks)**  
   - Ensure you can generate the merged DataFrame containing:  
     - OHLCV data  
     - Technical indicators  
     - Macro indicators  
   - Focus on getting **RSI** computed using **Code Snippets 8 and 9**.  
   - This process is essential and will help during the capstone project.

2. ⚠️ **IMPORTANT** Please use this file to solve the Home Assignment (**all next steps**)
 
   Download precomputed data using this snippet:

   ```python
   import gdown
   import pandas as pd

   file_id = "1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-"
   gdown.download(f"https://drive.google.com/uc?id={file_id}", "data.parquet", quiet=False)
   df = pd.read_parquet("data.parquet", engine="pyarrow")

3. **RSI Strategy Setup:**  
   - RSI is already available in the dataset as a field.  
   - The threshold for **oversold** is defined as `RSI < 25`.

4. **Filter the dataset by RSI and date:**  
   ```python
   rsi_threshold = 25
   selected_df = df[
       (df['rsi'] < rsi_threshold) &
       (df['Date'] >= '2000-01-01') &
       (df['Date'] <= '2025-06-01')
   ]
5. **Calculate Net Profit Over 25 Years:**  
   - Total number of trades: **1568**  
   - For each trade, you invest **$1000**  
   - Use the 30-day forward return (`growth_future_30d`) to compute net earnings:  
     ```python
     net_income = 1000 * (selected_df['growth_future_30d'] - 1).sum()
     ```

   - **Final Answer:**  
     What is the **net income in $K** (i.e., in thousands of dollars) that could be earned using this RSI-based oversold strategy from 2000–2025?


In [235]:
import gdown

In [236]:
file_id = "1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-"
gdown.download(f"https://drive.google.com/uc?id={file_id}", "data.parquet", quiet=False)
df = pd.read_parquet("data.parquet", engine="pyarrow")

Downloading...
From (original): https://drive.google.com/uc?id=1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-
From (redirected): https://drive.google.com/uc?id=1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-&confirm=t&uuid=dab5745c-7754-4ba1-8603-11a59342f023
To: /home/hachan/datatalksclub/stock-markets-analytics-zoomcamp-2025/data.parquet
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 130M/130M [00:03<00:00, 37.1MB/s]


In [247]:
df

,Open,High,Low,Close_x,Volume,Dividends,Stock Splits,Ticker,Year,Month,...,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d
0,0.054277,0.062259,0.054277,0.059598,1.031789e+09,0.0,0.0,MSFT,1986,1986-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.059598,0.062791,0.059598,0.061726,3.081600e+08,0.0,0.0,MSFT,1986,1986-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.061726,0.063323,0.061726,0.062791,1.331712e+08,0.0,0.0,MSFT,1986,1986-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.062791,0.063323,0.060662,0.061194,6.776640e+07,0.0,0.0,MSFT,1986,1986-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.061194,0.061726,0.059598,0.060130,4.789440e+07,0.0,0.0,MSFT,1986,1986-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5686,3615.800049,3672.500000,3608.399902,3648.699951,1.678934e+06,0.0,0.0,LT.NS,2025,2025-05-01,...,NaN,NaN,NaN,NaN,1.003714,1.020064,1.036306,1.156301,1.233323,1.597248
5687,3648.699951,3665.000000,3603.000000,3640.000000,2.013954e+06,0.0,0.0,LT.NS,2025,2025-05-01,...,0.993181,0.989654,0.781299,0.842957,0.995927,1.011165,1.020634,1.162549,1.292217,1.570651
5688,3660.000000,3663.000000,3620.000000,3646.300049,1.293244e+06,0.0,0.0,LT.NS,2025,2025-05-01,...,0.992203,1.000308,0.798376,0.886128,0.989061,0.988691,0.982898,1.135015,1.272691,1.578452
5689,3663.899902,3668.899902,3618.000000,3655.300049,1.972248e+06,0.0,0.0,LT.NS,2025,2025-05-01,...,0.978792,0.991959,0.794034,0.863857,0.979958,0.965291,0.945990,1.120454,1.252080,1.563254


In [242]:
df['rsi']

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
          ...    
5686    68.312972
5687    67.123448
5688    67.563912
5689    68.218963
5690    69.670158
Name: rsi, Length: 229932, dtype: float64

In [243]:
rsi_threshold = 25
selected_df = df[
    (df['rsi'] < rsi_threshold) &
    (df['Date'] >= '2000-01-01') &
    (df['Date'] <= '2025-06-01')
]

In [244]:
net_income = 1000 * (selected_df['growth_future_30d'] - 1).sum()

In [245]:
net_income

np.float64(24295.523125248386)

In [246]:
selected_df['growth_future_30d']

3668    0.985394
3669    0.972222
3680    0.964641
3681    1.005637
3682    0.981941
          ...   
4405    1.007871
4406    0.981580
4407    1.118479
4408    1.142111
4409    1.011231
Name: growth_future_30d, Length: 1568, dtype: float64